In [202]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.models import Model
import warnings 
warnings.filterwarnings('ignore')
import tensorflow as tf
print(tf.__version__)

2.7.0


In [203]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
train.head()

,image,label
0,glacier/59804579.jpg,glacier
1,glacier/15264675.jpg,glacier
2,mountain/11008530.jpg,mountain
3,sea/33613868.jpg,sea
4,buildings/84069666.jpg,buildings


In [204]:
train["label"] = train["label"].replace({0: "buildings",1: "forest",2: "glacier",3: "mountain",4: "sea",5: "street"})

In [205]:
import gc
gc.collect()

27110

In [235]:
test

,image
0,70003155.jpg
1,14715617.jpg
2,13082179.jpg
3,97804376.jpg
4,14111240.jpg
...,...
2994,16813071.jpg
2995,15337171.jpg
2996,11069450.jpg
2997,87101838.jpg


In [206]:
train_df, validate_df = train_test_split(train, test_size=0.20, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [207]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,)

In [215]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 35
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "./train/", 
    x_col='image',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE)

validation_generator = train_datagen.flow_from_dataframe(
    validate_df, 
    "./train/",
    x_col='image',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE)

Found 11227 validated image filenames belonging to 6 classes.
Found 2807 validated image filenames belonging to 6 classes.


In [220]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    test,
    "./test/",
    x_col = 'image',
    y_col = None,
    target_size = IMAGE_SIZE,
    class_mode=None,
    batch_size=BATCH_SIZE
)

Found 2999 validated image filenames.


In [219]:
test

,image
0,70003155.jpg
1,14715617.jpg
2,13082179.jpg
3,97804376.jpg
4,14111240.jpg
...,...
2994,16813071.jpg
2995,15337171.jpg
2996,11069450.jpg
2997,87101838.jpg


In [216]:
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.0000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False)
    return lr_callback

In [217]:
count = 4
model = [0]*count
for i in range(count):
    base_model =  tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    add_model = Sequential()
    add_model.add(GlobalAveragePooling2D(input_shape = base_model.output_shape[1:]))
    add_model.add(Dropout(0.3))
    add_model.add(Dense(512, activation = 'relu'))
    add_model.add(Dense(6, activation = 'softmax'))

    model[i] = Model(inputs=base_model.input, outputs=add_model(base_model.output))

    model[i].compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  metrics=['accuracy'])
    

In [218]:
history = [0]*count
for j in range(count):
    print(f'{j+1} Model')
    history[j] = model[j].fit(train_generator,
                                epochs=120,
                                validation_data=validation_generator,
                                batch_size=BATCH_SIZE,
                                callbacks=[get_lr_callback()],
                                verbose = 1)
    print(f'{j} Model  Train Accuracy = {2:.5f}, Validation Accuracy = {2:.5f}', max(history[j].history['acc']), max(history[j].history['val_acc']))

1 Model
Epoch 1/120
321/321 [==============================] - 93s 270ms/step - loss: 1.8167 - accuracy: 0.1806 - val_loss: 1.7994 - val_accuracy: 0.2412 - lr: 1.0000e-06
Epoch 2/120
321/321 [==============================] - 87s 269ms/step - loss: 1.6731 - accuracy: 0.3185 - val_loss: 1.7257 - val_accuracy: 0.2608 - lr: 4.3000e-06
Epoch 3/120
321/321 [==============================] - 87s 271ms/step - loss: 1.3649 - accuracy: 0.5162 - val_loss: 1.3989 - val_accuracy: 0.5048 - lr: 7.6000e-06
Epoch 4/120
321/321 [==============================] - 88s 274ms/step - loss: 0.9827 - accuracy: 0.6515 - val_loss: 1.1241 - val_accuracy: 0.5957 - lr: 1.0900e-05
Epoch 5/120
321/321 [==============================] - 87s 271ms/step - loss: 0.7388 - accuracy: 0.7334 - val_loss: 1.0942 - val_accuracy: 0.5593 - lr: 1.4200e-05
Epoch 6/120
321/321 [==============================] - 88s 274ms/step - loss: 0.5914 - accuracy: 0.7871 - val_loss: 1.0331 - val_accuracy: 0.5989 - lr: 1.7500e-05
Epoch 7/120
32

Epoch 51/120
321/321 [==============================] - 89s 278ms/step - loss: 0.3088 - accuracy: 0.8910 - val_loss: 0.3467 - val_accuracy: 0.8785 - lr: 1.0007e-06
Epoch 52/120
321/321 [==============================] - 91s 282ms/step - loss: 0.3118 - accuracy: 0.8888 - val_loss: 0.3529 - val_accuracy: 0.8807 - lr: 1.0006e-06
Epoch 53/120
321/321 [==============================] - 90s 279ms/step - loss: 0.3053 - accuracy: 0.8902 - val_loss: 0.3429 - val_accuracy: 0.8803 - lr: 1.0005e-06
Epoch 54/120
321/321 [==============================] - 89s 275ms/step - loss: 0.3140 - accuracy: 0.8878 - val_loss: 0.3498 - val_accuracy: 0.8796 - lr: 1.0004e-06
Epoch 55/120
321/321 [==============================] - 91s 282ms/step - loss: 0.3068 - accuracy: 0.8894 - val_loss: 0.3454 - val_accuracy: 0.8853 - lr: 1.0003e-06
Epoch 56/120
321/321 [==============================] - 90s 279ms/step - loss: 0.3034 - accuracy: 0.8922 - val_loss: 0.3472 - val_accuracy: 0.8760 - lr: 1.0002e-06
Epoch 57/120
321

Epoch 101/120
321/321 [==============================] - 89s 276ms/step - loss: 0.2599 - accuracy: 0.9075 - val_loss: 0.3173 - val_accuracy: 0.8910 - lr: 1.0000e-06
Epoch 102/120
321/321 [==============================] - 89s 277ms/step - loss: 0.2686 - accuracy: 0.9055 - val_loss: 0.3182 - val_accuracy: 0.8924 - lr: 1.0000e-06
Epoch 103/120
321/321 [==============================] - 89s 278ms/step - loss: 0.2652 - accuracy: 0.9053 - val_loss: 0.3246 - val_accuracy: 0.8849 - lr: 1.0000e-06
Epoch 104/120
321/321 [==============================] - 89s 278ms/step - loss: 0.2595 - accuracy: 0.9080 - val_loss: 0.3204 - val_accuracy: 0.8888 - lr: 1.0000e-06
Epoch 105/120
321/321 [==============================] - 89s 277ms/step - loss: 0.2589 - accuracy: 0.9075 - val_loss: 0.3148 - val_accuracy: 0.8928 - lr: 1.0000e-06
Epoch 106/120
321/321 [==============================] - 90s 279ms/step - loss: 0.2605 - accuracy: 0.9057 - val_loss: 0.3295 - val_accuracy: 0.8888 - lr: 1.0000e-06
Epoch 107/

KeyError: 'acc'

In [221]:
results = np.zeros((test.shape[0], 6))
results = results + model[0].predict_generator(test_generator)

In [224]:
indices = np.argmax(results, axis = 1)

In [232]:
labels = (train_generator.class_indices)
labels = dict((v, k) for k, v in labels.items())
predictions = [labels[k] for k in indices]

In [237]:
submission = pd.DataFrame({
    "image" : test['image'],
    'label' : predictions
})

submission.to_csv('tfs.csv', index = False)